In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression


from scipy.optimize import minimize

import time

#### Load dataset

In [2]:
start_loading_data_time = time.time()
data = pd.read_csv('dataset/train.csv')
print('Data loading time: ' + str(time.time() - start_loading_data_time))

print(data.columns)

X = data.iloc[:, 1:].values
y = data['label'].values

Data loading time: 2.955108165740967
Index(['label', 'pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5',
       'pixel6', 'pixel7', 'pixel8',
       ...
       'pixel774', 'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779',
       'pixel780', 'pixel781', 'pixel782', 'pixel783'],
      dtype='object', length=785)


In [3]:
m = X.shape[0]
n = X.shape[1]
ones = np.ones(shape = (m, 1))

# Adding bias to the training dataset 
X_biased = np.concatenate([ones, X], axis = 1)
print(m, n)

42000 784


### Multiclass Classification

#### Logistic regression hypothesis
#### $$ h_{\theta}(x) = g(\theta^{T}x)$$
#### $$ g(z)=\frac{1}{1+e^{−z}} $$

In [4]:
def sigmoid(z):
    return(1 / (1 + np.exp(-z)))

#### Regularized Cost Function 
#### $$ J(\theta) = \frac{1}{m}\sum_{i=1}^{m}\big[-y^{(i)}\, log\,( h_\theta\,(x^{(i)}))-(1-y^{(i)})\,log\,(1-h_\theta(x^{(i)}))\big] + \frac{\lambda}{2m}\sum_{j=1}^{n}\theta_{j}^{2}$$
#### Vectorized Cost Function
#### $$ J(\theta) = \frac{1}{m}\big((\,log\,(g(X\theta))^Ty+(\,log\,(1-g(X\theta))^T(1-y)\big) + \frac{\lambda}{2m}\sum_{j=1}^{n}\theta_{j}^{2}$$

In [5]:
def lrcostFunctionReg(theta, reg, X, y):
    m = y.size
    h = sigmoid(X.dot(theta))
    
    J = -1*(1/m)*(np.log(h).T.dot(y)+np.log(1-h).T.dot(1-y)) + (reg/(2*m))*np.sum(np.square(theta[1:]))
    
    if np.isnan(J[0]):
        return(np.inf)
    return(J[0])    

In [6]:
def lrgradientReg(theta, reg, X,y):
    m = y.size
    h = sigmoid(X.dot(theta.reshape(-1,1)))
      
    grad = (1/m)*X.T.dot(h-y) + (reg/m)*np.r_[[[0]],theta[1:].reshape(-1,1)]
        
    return(grad.flatten())

#### One-vs-all Classification

In [7]:
def oneVsAll(features, classes, n_labels, reg):
    initial_theta = np.zeros((features.shape[1],1))  # 401x1
    all_theta = np.zeros((n_labels, X.shape[1])) #10x401

    for c in np.arange(1, n_labels+1):
        res = minimize(lrcostFunctionReg, initial_theta, args=(reg, features, (classes == c)*1), method=None,
                       jac=lrgradientReg, options={'maxiter':50})
        all_theta[c-1] = res.x
    return(all_theta)

In [8]:
start_train_time = time.time()
theta = oneVsAll(X_biased, y, 10, 0.1)
print('Training time: ' + str(time.time() - start_train_time))

MemoryError: 

#### One-vs-all Prediction

In [ ]:
def predictOneVsAll(all_theta, features):
    probs = sigmoid(X.dot(all_theta.T))
        
    # Adding one because Python uses zero based indexing for the 10 columns (0-9),
    # while the 10 classes are numbered from 1 to 10.
    return(np.argmax(probs, axis=1)+1)

### Evaluation on test set

In [9]:
X_test = pd.read_csv('dataset/test.csv').iloc[:, 1:].values
y_test = pd.read_csv('dataset/test.csv').iloc[:, 0].values

In [ ]:
pred = predictOneVsAll(theta, X_test)
print('Training set accuracy: {} %'.format(np.mean(pred == y.ravel())*100))

#### Multiclass Logistic Regression with scikit-learn

In [ ]:
clf = LogisticRegression(C=10, penalty='l2', solver='liblinear')
# Scikit-learn fits intercept automatically, so we exclude first column with 'ones' from X when fitting.
print(X.shape, y.shape)

start_train_time = time.time()
clf.fit(X, y)
print('Train time with sklearn: ' + str(time.time() - start_train_time))

(42000, 784) (42000,)


In [ ]:
pred2 = clf.predict(X_test)
print('Test set accuracy: {} %'.format(np.mean(pred2 == y_test)*100))

In [ ]:
from sklearn.metrics import confusion_matrix

conf_mat = confusion_matrix(y_test, pred2)

conf_mat